In [1]:
%load_ext autoreload
%autoreload 2
from models import NNDynamicModel, MPCcontroller
import tensorflow as tf
import numpy as np
import json

/Users/loulinhui/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
regularizer = tf.contrib.layers.l2_regularizer(scale=1e-9)
activation = tf.nn.relu
output_activation = None
learning_rate = 0.003
batch_size = 128

In [3]:
# load data
with open('./sample.json', 'r') as f:
    data = json.loads(f.read())
    
print('loaded %d data.' % (len(data)))


loaded 8000 data.


In [4]:
obs = [d[0] for d in data]
normalization = [np.min(obs, axis=0), np.max(obs, axis=0), np.mean(obs, axis=0)]
print(normalization)

session = tf.Session()

[array([  0.   , -50.   ,  90.   ,  17.   ,   6.166]), array([  0.   , 625.   , 105.   ,  51.   ,   6.657]), array([  0.        , 424.27775   ,  97.558125  ,  29.69      ,
         6.21954263])]


In [5]:
model = NNDynamicModel(
    name='test',
    n_layers=3,
    size=128,
    activation=activation,
    output_activation=output_activation,
    kernel_regularizer=regularizer,
    normalization=normalization,
    batch_size=batch_size,
    iterations=40,
    learning_rate=learning_rate,
    sess=session
)

In [6]:
init = tf.global_variables_initializer()
session.run(init)

In [7]:
model.fit(data)

on iter_step 0, loss = 0.1205008
on iter_step 1, loss = 0.0937012
on iter_step 2, loss = 0.0847356
on iter_step 3, loss = 0.0813896
on iter_step 4, loss = 0.0793179
on iter_step 5, loss = 0.0776664
on iter_step 6, loss = 0.0768987
on iter_step 7, loss = 0.0771575
on iter_step 8, loss = 0.0760348
on iter_step 9, loss = 0.0746681
on iter_step 10, loss = 0.0750619
on iter_step 11, loss = 0.0743006
on iter_step 12, loss = 0.0739106
on iter_step 13, loss = 0.0745490
on iter_step 14, loss = 0.0727311
on iter_step 15, loss = 0.0738017
on iter_step 16, loss = 0.0726978
on iter_step 17, loss = 0.0723282
on iter_step 18, loss = 0.0716955
on iter_step 19, loss = 0.0727883
on iter_step 20, loss = 0.0717801
on iter_step 21, loss = 0.0717848
on iter_step 22, loss = 0.0712575
on iter_step 23, loss = 0.0706263
on iter_step 24, loss = 0.0718270
on iter_step 25, loss = 0.0698642
on iter_step 26, loss = 0.0708387
on iter_step 27, loss = 0.0697444
on iter_step 28, loss = 0.0705103
on iter_step 29, loss = 

In [8]:
mpc = MPCcontroller(dyn_model=model) 

In [21]:
nd = [d for d in data if d[-1] < 0]
obs, action, nobs, reward = nd[0]
mobs, mreward = model.predict([obs], [action])
print(mobs, mreward)
print(obs, action, nobs, reward)
print('cha', np.sum(obs - mobs))
action = mpc.get_action(obs)
print(action)

[[1.4527377e-02 1.7985365e+01 1.0382662e+02 1.7306196e+01 6.7099104e+00]] [-0.59361744]
[0.0, 106.0, 105.0, 17.0, 6.649000000000217] [2.0, 0.8178347961300148] [0.0, 51.0, 105.0, 17.0, 6.663000000000221] -1
cha 88.80637909451148
[1.         0.03801776]


In [10]:
from game_controller import GameEnv

In [11]:
env = GameEnv()
env.start()

obs = None
step = 1000
while step > 0:
    if env.status == 'CRASHED':
        env.restart()
    if obs is None:
        obs = env.get_observation()
    if obs is not None:
        action = mpc.get_action(obs)
        obs, _, _ = env.perform_action(action)
        step -= 1

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=65.0.3325.181)
  (Driver info: chromedriver=2.37.544337 (8c0344a12e552148c185f7d5117db1f28d6c9e85),platform=Mac OS X 10.13.4 x86_64)
